In [1]:
import numpy as np
import torch

In [2]:
from matplotlib import pyplot as plt

In [3]:
from sklearn.linear_model import LogisticRegressionCV

In [4]:
from tqdm.auto import tqdm

In [5]:
import kcmc
from kcmc.estimators import confounding_robust_estimator, hajek, ipw
from kcmc.data import generate_data_binary as generate_data
from kcmc.data import evaluate_policy_binary as evaluate_policy

### Synthetic Data from Kallus 2018

Notes on data types:

- Only use torch tensor for `Y` and `r`
- For other data, use numpy array

In [8]:
beta0 = 2.5
beta0_t = -2
beta_x = np.asarray([0, .5, -0.5, 0, 0])
beta_x_t = np.asarray([-1.5, 1, -1.5, 1., 0.5])
beta_xi = 1
beta_xi_t = -2
beta_e_x = np.asarray([0, .75, -.5, 0, -1])
mu_x = np.asarray([-1, .5, -1, 0, -1]);

def generate_data_continuous(n):
    xi = (np.random.rand(n) > 0.5).astype(int)
    X = mu_x[None, :] + np.random.randn(n * 5).reshape(n, 5)
    eps = [np.random.randn(n) for t in (0, 1)]
    Y = np.array([
        X @ (beta_x + beta_x_t * t) + (beta_xi + beta_xi_t * t) * xi + (beta0 + beta0_t * t) + eps[t]
        for t in (0, 1)
    ])
    U = (Y[0, :] > Y[1, :]).astype(int)
    z = X @ beta_e_x
    e_x = np.exp(z) / (1 + np.exp(z))
    mu_xu = (6 * e_x) / (4 + 5 * U + e_x * (2 - 5 * U))
    mu_xu = np.clip(mu_xu, 1e-6, 1 - 1e-6)
    unif = np.random.rand(n)
    a, b = 4 * mu_xu + 1, 4 * (1 - mu_xu) + 1
    T = (1 - ber) * beta.rvs(a, b) + ber * unif
    e_xu = beta.pdf(T, a, b)
    Y = (1 - T) * Y[0, :] + T * Y[1, :]
    Y = torch.as_tensor(Y)
    return Y, T, X, U, e_x, e_xu

def evaluate_policy_continuous(policy, n=1000, requires_grad=False):
    xi = (np.random.rand(n) > 0.5).astype(int)
    X = mu_x[None, :] + np.random.randn(n * 5).reshape(n, 5)
    eps = [np.random.randn(n) for t in (0, 1)]
    Y = np.array([
        X @ (beta_x + beta_x_t * t) + (beta_xi + beta_xi_t * t) * xi + (beta0 + beta0_t * t) + eps[t]
        for t in (0, 1)
    ])
    T = policy(X).rsample() if requires_grad else policy(X).sample()
    Y = torch.as_tensor(Y) if isinstance(T, torch.Tensor) else Y
    Y = (1 - T) * Y[0, :] + T * Y[1, :]
    return Y.mean()

In [6]:
n = 1000
np.random.seed(0)
Y, T, X, U, e_x, e_xu = generate_data(n)

In [7]:
beta_e_x = np.asarray([0, .75, -.5, 0, -1])

def toy_policy(X):
    z = torch.as_tensor(X) @ torch.as_tensor(beta_e_x)
    e_x = torch.exp(z) / (1 + torch.exp(z))
    return torch.stack([e_x, 1. - e_x]).T

# def zero_policy(X):
#    return np.zeros(X.shape[0], dtype=int)

### Estimate Propensity Score (Conditional Density Estimation)

#### Discrete Case

In [8]:
# Logistic regression is already quite good

model = LogisticRegressionCV().fit(X, T)
e_xx = model.predict_proba(X)[range(n), T]

np.mean(np.abs(e_xx - e_xu))

0.04417433219435932

In [9]:
p_t = e_xx

### Ground Truth

In [10]:
evaluate_policy(toy_policy, n=1000000)

tensor(3.6258, dtype=torch.float64)

### IPW estimator

In [11]:
est_ipw = ipw(Y, T, X, p_t, toy_policy)

In [12]:
est_ipw

tensor(3.4380, dtype=torch.float64)

### Hajek estimator

In [13]:
est_hajek = hajek(Y, T, X, p_t, toy_policy)

In [14]:
est_hajek

tensor(3.7397, dtype=torch.float64)

### Implement Confounding Robust Inference

In [15]:
kcmc.estimators.f_divergences

['KL',
 'inverse KL',
 'Jensen-Shannon',
 'squared Hellinger',
 'Pearson chi squared',
 'Neyman chi squared',
 'total variation']

In [16]:
est, w = confounding_robust_estimator(
    Y, T, X, p_t, toy_policy,
    D=200,
    lambd=1.5, 
    gamma=0.01,
    alpha=0.05,
    hard_kernel_const=False,
    f_divergence='total variation', 
    hajek_const=True,
    kernel_const=False,
    quantile_const=False,
    tan_box_const=True,
    lr_box_const=False,
    f_const=False,
    return_w=True
)

In [17]:
est

tensor(3.2750, dtype=torch.float64)

In [18]:
est_hajek

tensor(3.7397, dtype=torch.float64)

#### Select Kernel for KCMC

In [19]:
# Guessing kernel with approximate solution
_, w_guess = confounding_robust_estimator(
    Y, T, X, p_t, toy_policy, lambd=1.5,
    hajek_const=True, tan_box_const=True, return_w=True,
)
e_guess = p_t * w_guess - 1
gp_kernel = kcmc.estimators.fit_gp_kernel(e_guess, T, X)
sigma2 = gp_kernel.k1.noise_level
kernel = gp_kernel.k2

In [20]:
est = confounding_robust_estimator(
    Y, T, X, p_t, toy_policy,
    D=200,
    lambd=1.5, 
    gamma=0.5,
    alpha=0.05,
    sigma2=sigma2,
    kernel=kernel,
    hard_kernel_const=False,
    f_divergence='KL', 
    hajek_const=True,
    kernel_const=False,
    quantile_const=True,
    tan_box_const=True,
    lr_box_const=False,
    f_const=False,
)

NameError: name 'pi_np' is not defined

In [ ]:
est

### Compare Policy Learning Performance

In [40]:
def base_policy(X):
    return 0.5 * torch.ones((X.shape[0], 2), dtype=float)

def LR_policy(X, beta):
    X = np.concatenate([np.ones([X.shape[0], 1]), X], axis=1)
    p = torch.sigmoid(torch.tensor(X) @ beta)
    return torch.stack([p, 1.0 - p]).T

In [41]:
class nnPolicy(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.nn = torch.nn.Sequential(
            torch.nn.Linear(5, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 32),
            torch.nn.ReLU(),
            torch.nn.Linear(32, 1),
            torch.nn.Sigmoid(),
        )
        self.nn[-2].weight.data[:] = 0.
        self.nn[-2].bias.data[:] = 0.

    def forward(self, X):
        X = torch.as_tensor(X, dtype=torch.float32)
        p = self.nn(X)[:, 0]
        return torch.stack([p, 1.0 - p]).T

In [42]:
evaluate_policy(base_policy, n=100000)

tensor(3.7457, dtype=torch.float64)

In [43]:
nn_policy = nnPolicy()
evaluate_policy(nn_policy, n=100000)

tensor(3.7425, dtype=torch.float64, grad_fn=<MeanBackward0>)

#### Best Policy

In [52]:
train_steps = 101
beta_best = torch.zeros(6, requires_grad=True, dtype=float)
best_policy = lambda X: LR_policy(X, beta_best)
optimizer = torch.optim.RMSprop(lr=1e-1, params=[beta_best])
pbar = tqdm(range(train_steps))
for i in pbar:
    value = evaluate_policy(best_policy, n=100000)
    (- value).backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Value: {value}")

  0%|          | 0/101 [00:00<?, ?it/s]

#### Just Run Hajek optimization

In [55]:
train_steps = 101
beta_ipw = torch.zeros(6, requires_grad=True, dtype=float)
hajek_policy = lambda X: LR_policy(X, beta_ipw)
optimizer = torch.optim.RMSprop(lr=1e-1, params=[beta_ipw])
pbar = tqdm(range(train_steps))
for i in pbar:
    value = hajek(Y, T, X, p_t, hajek_policy)
    (- value).backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Value: {value}")

  0%|          | 0/101 [00:00<?, ?it/s]

In [56]:
evaluate_policy(ipw_policy, n=100000)

tensor(4.6807, dtype=torch.float64, grad_fn=<MeanBackward0>)

### Just run min-max optimzation

In [78]:
def evaluate_confounding_robust_hajek(policy):
    est = confounding_robust_estimator(
        Y, T, X, p_t, policy, 
        lambd=1.5, hajek_const=True, tan_box_const=True,
    )
    return est

In [79]:
def evaluate_confounding_robust_kernel(policy):
    est = confounding_robust_estimator(
        Y, T, X, p_t, policy, 
        D=200, lambd=1.5, alpha=0.05, 
        sigma2=sigma2, kernel=kernel,
        kernel_const=True,
        tan_box_const=True,
    )
    return est

In [80]:
# Hajek policy min-max value
train_steps = 101
beta_hajek = torch.zeros(6, requires_grad=True, dtype=float)
hajek_policy = lambda X: LR_policy(X, beta_hajek)
optimizer = torch.optim.SGD(lr=1e-1, params=[beta_hajek])
pbar = tqdm(range(train_steps))
for i in pbar:
    hajek_value = evaluate_confounding_robust_hajek(hajek_policy)
    (- hajek_value).backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Value: {hajek_value}")

  0%|          | 0/101 [00:00<?, ?it/s]

In [81]:
evaluate_policy(hajek_policy, n=100000)

tensor(4.4220, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [82]:
evaluate_confounding_robust_kernel(hajek_policy)

tensor(4.0741, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [64]:
# Kernel policy min-max value
train_steps = 101
beta_kernel = torch.zeros(6, requires_grad=True, dtype=float)
kernel_policy = lambda X: LR_policy(X, beta_kernel)
optimizer = torch.optim.RMSprop(lr=1e-1, params=[beta_kernel])
pbar = tqdm(range(train_steps))
for i in pbar:
    kernel_value = evaluate_confounding_robust_kernel(kernel_policy)
    (- kernel_value).backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Value: {kernel_value}")

  0%|          | 0/101 [00:00<?, ?it/s]

In [68]:
pbar = tqdm(range(train_steps))
for i in pbar:
    kernel_value = evaluate_confounding_robust_kernel(kernel_policy)
    (- kernel_value).backward()
    optimizer.step()
    optimizer.zero_grad()
    pbar.set_description(f"Value: {kernel_value}")

  0%|          | 0/101 [00:00<?, ?it/s]

In [69]:
evaluate_policy(kernel_policy, n=100000)

tensor(4.6475, dtype=torch.float64, grad_fn=<MeanBackward0>)

In [70]:
evaluate_confounding_robust_kernel(kernel_policy)

tensor(4.4631, dtype=torch.float64, grad_fn=<MeanBackward0>)

### Conclusions
- The difference in performance is sample-dependent
  - If the sample is well confounded, then kernel + hajek tends to be better than hajek
- IPW estimator is good enough in most cases, so we might not actually need confounding-robust methods
- Still, kernel + hajek method offers better lower-bound of the policy value, so it must have some use cases.

### Make a Table for Slide

In [ ]:
pi1 = lambda x: 0.5
pi2 = toy_policy
pi3 = best_policy
pi4 = hajek_policy
pi5 = kernel_policy
pi6 = ipw_policy

In [ ]:
table = {}
for i, pi in tqdm.tqdm(enumerate([pi1, pi2, pi3, pi4, pi5, pi6])):
    table[f"$\\pi_{i+1}$"] = row = {}
    row['$V_\\text{inf}$'] = float(evaluate_policy(pi, n=100000))
    prob = pi(X) * torch.tensor(T) + (1 - pi(X)) * torch.tensor(1 - T)
    r = torch.tensor(Y) * prob
    row['$\\hat V_\\text{inf}^\\Hajek$'] = float(confoundingRobustHajek(r, T, a, b))
    row['$\\hat V_\\text{inf}^\\kernel$'] = float(confoundingRobustKernel(r, T, X, a, b, p_t))
    row['$\\hat V_\\text{inf}^\\text{IPW}$'] = float(IPW(r, p_t))

In [ ]:
table_copy = table.copy()
table.pop("$\\pi_6$")

In [ ]:
print("\\begin{tabular}{ | m{4em} || " + "m{3em} " * len(table) + "| }")
print("\\hline")
header = "{}\t" + "&{}\t" * len(table) + "\\\\"
print(header.format("", *table.keys()))
print("\\hline")

row_names = table["$\\pi_1$"].keys()
for row in row_names:
    s = "{}\t" + "&{:1.3f}\t" * len(table) + "\\\\"
    print(s.format(row, *[col[row] for col in table.values()]))
    print("\\hline")
print("\\end{tabular}")


### Estimate Propensity Score (Conditional Density Estimation)
- This requires consistent nominal propensity score (p_obs(t|x))
- For discrete t, just run kernel logistic regressionregret_kernel

#### Discrete Case

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
# from sklearn.metrics import roc_auc_score, make_scorer
# roc_auc_score = make_scorer(roc_auc_score, greater_is_better=True, needs_proba=True)

model = LogisticRegressionCV().fit(X, T)
e_xx = model.predict_proba(X)[:, 1]

np.mean(np.abs(e_xx - e_x))

In [ ]:
import optuna
from sklearn.decomposition import KernelPCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

def objective(trial):
    gamma = trial.suggest_float('gamma', 1e-2, 1e+2, log=True)
    lambd = trial.suggest_float('lambd', 1e-3, 1e+3, log=True)
    model = Pipeline([
        ('kpca', KernelPCA(n_components=100, kernel='rbf', gamma=gamma)),
        ('LogReg', LogisticRegression(C=1/lambd)),
    ])   
    cv_scores = cross_val_score(model, X, y=T, cv=4)
    return cv_scores.mean()

In [ ]:
study = optuna.create_study(direction='maximize')
study.optimize(func=objective, n_trials=100)

In [ ]:
study.best_params

In [ ]:
Z = KernelPCA(n_components=200, kernel='rbf', gamma=0.01).fit_transform(X)
model = LogisticRegression(C=5).fit(Z, T)
e_xx = model.predict_proba(Z)[:, 1]

np.mean(np.abs(e_xx - e_x))

In [ ]:
np.mean(1 / e_x * T), np.mean(1 / (1 - e_x) * (1 - T)), # both should be 1

In [ ]:
np.mean(1 / e_xx * T), np.mean(1 / (1 - e_xx) * (1 - T)), # both should be 1

#### Continuous Case (Maybe on the next notebook)

In [ ]:
from statsmodels.nonparametric.kernel_density import KDEMultivariateConditional

In [ ]:
EPS = 1e-6

def benchmark_bandwidth(bw, T, X, cv=5):
    n = T.shape[0]
    indep_type='c' * X.shape[1]
    dep_type='u'
    split = (np.arange(n) % 5)
    cross_entropy = 0
    for i in range(cv):
        train = (split != i)
        test = (split == i)
        model = KDEMultivariateConditional(T[train, None], X[train, :], dep_type='c', indep_type='c' * X.shape[1], bw=bw)
        cross_entropy += np.mean(model.pdf(T[test, None], X[test, :]) + EPS) / cv
    return cross_entropy

In [ ]:
best_scale = 0
best_score = - float('inf')
for scale in [0.1, 0.2, 0.3, 0.5, 0.7, 1, 1.5, 2, 3, 5]:
    bw = scale * np.concatenate([[Y.std()], X.std(axis=0)]) / Y.shape[0] ** (1 / 7)
    score = benchmark_bandwidth(bw, Y, X)
    if score > best_score:
        best_scale = scale
        best_score = score

In [ ]:
best_scale, best_score

In [ ]:
bw0 = best_scale * np.concatenate([[Y.std()], X.std(axis=0)]) / Y.shape[0] ** (1 / 7)
model = KDEMultivariateConditional(Y[:, None], X, dep_type='c', indep_type='c' * X.shape[1], bw=bw0)

In [ ]:
e_xx = model.pdf(Y[:, None], X)
e_xx

In [ ]:
plt.hist(e_xx)

In [ ]:
model.bw

In [ ]:
[np.mean(1. / e_xx * ((i <= Y) & (Y <= i + 1))) for i in range(7)]  # Should go to 1

In [ ]:
# Shouldn't we use loo estimate for propensity? (For reducing the sampling bias)

In [ ]:
model_loocv = KDEMultivariateConditional(Y[:500, None], X[:500], dep_type='c', indep_type='c' * X.shape[1], bw='cv_ml')
model_loocv = KDEMultivariateConditional(Y[:, None], X, dep_type='c', indep_type='c' * X.shape[1], bw=model_loocv.bw)
model_loocv.pdf(T[:, None], X)

In [ ]:
model_loocv.bw

In [ ]:
e_xx = model_loocv.pdf(Y[:, None], X)
e_xx

In [ ]:
[np.mean(1. / e_xx * ((i <= Y) & (Y <= i + 1))) for i in range(7)]  # Should go to 1

In [ ]:
Y, T, X, U, e_x, e_xu = generate_data(2000)

In [ ]:
# Maybe better to use gaussian process regression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import ConstantKernel, WhiteKernel, RBF
from scipy.stats import norm

kernel = WhiteKernel() + ConstantKernel() * RBF()
model = GaussianProcessRegressor(kernel=kernel).fit(X, Y)
mu, sigma = model.predict(X, return_std=True)
e_xx = norm.pdf(Y, loc=mu, scale=sigma)

In [ ]:
plt.hist(e_xx)

In [ ]:
[np.mean(1. / e_xx * ((i <= Y) & (Y <= i + 1))) for i in range(7)]  # Should go to 1